运行main函数即可

In [ ]:
def main():
    import os
    import shutil
    import cv2
    import random
    from random import choice
    from imgaug import augmenters as iaa
    import imgaug as ia
    import keras
    from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten,BatchNormalization,Activation,AveragePooling2D,Concatenate
    from keras.models import Model
    from keras.preprocessing import image
    import numpy as np
    import pandas as pd
    from keras.models import load_model  
    from keras.layers import GlobalAveragePooling2D,Flatten
    from keras.optimizers import Adam,SGD
    from keras.preprocessing.image import ImageDataGenerator
    from keras.callbacks import ModelCheckpoint
    import matplotlib.pyplot as plt
    from keras.applications import inception_resnet_v2
    from keras.models import load_model
    import keras.backend as backend
    import time
    import csv
    import sys
    
    #创建normal和bad文件夹 normal 1316张 bad 706张
    if not os.path.isdir("../data/normal"):
        os.mkdir("../data/normal")
    if not os.path.isdir("../data/bad"):
        os.mkdir("../data/bad")
    for i in os.listdir("../data/"):
        if i[-3:] != 'zip' and "test" not in i and i!='bad' and i!= 'normal':
            for j in os.listdir("../data/" + i):
                if(j == "正常"):
                    for m in os.listdir("../data/" + i + "/" + j):
                        if m[-3:] == 'jpg':
                            shutil.copy("../data/" + i + "/" + j + "/" + m,"../data/normal/")
                else:
                    for m in os.listdir("../data/" + i + "/" + j):
                        if m[-3:] == 'jpg':
                            shutil.copy("../data/" + i + "/" + j + "/" + m,"../data/bad/")
                            
    #划分出验证集
    val_list_bad = random.sample(os.listdir("../data/bad"),81)
    val_list_normal = random.sample(os.listdir("../data/normal"),132)

    if not os.path.isdir("../data/val"):
        os.mkdir("../data/val")
    if not os.path.isdir("../data/val/normal"):
        os.mkdir("../data/val/normal")
    if not os.path.isdir("../data/val/bad"):
        os.mkdir("../data/val/bad")
    for i in val_list_bad:
        shutil.copy("../data/bad/"+i,"../data/val/bad/")
    for i in val_list_normal:
        shutil.copy("../data/normal/"+i,"../data/val/normal/") 
        
    #对训练集里面的每张图进行resize 
    if not os.path.isdir("../data/normal_68"):
        os.mkdir("../data/normal_68")
    if not os.path.isdir("../data/bad_68"):
        os.mkdir("../data/bad_68")

    for i in os.listdir("../data/normal/"): 
        path = "../data/normal/" + i
        img = cv2.imread(path)
        img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)
        cv2.imwrite('../data/normal_68/' + i, img)

    for i in os.listdir("../data/bad/"): 
        path = "../data/bad/" + i
        img = cv2.imread(path)
        img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)
        cv2.imwrite('../data/bad_68/' + i, img)
        
    if not os.path.isdir("../data/train"):
        os.mkdir("../data/train")
    if not os.path.isdir("../data/train/bad"):
        os.mkdir("../data/train/bad")
    if not os.path.isdir("../data/train/normal"):
        os.mkdir("../data/train/normal")

    for i in os.listdir("../data/bad_68/"):
        shutil.copy("../data/bad_68/"+i,"../data/train/bad/")
    for i in os.listdir("../data/normal_68/"):
        shutil.copy("../data/normal_68/"+i,"../data/train/normal/")


    #进行aug
    #增强到5000
    TRAIN_DIR = "../data/bad_68/"
    TRAIN_AUG_DIR = "../data/train/bad/"


    seq = iaa.Sequential([
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                iaa.Multiply((0.9, 1.1)),
            ])

    normal_list = os.listdir(TRAIN_AUG_DIR)

    k = len(normal_list)
    imageList = []
    m = k
    while m < 5000:
        image = cv2.imread(TRAIN_AUG_DIR + choice(normal_list))
        imageList.append(image)
        m += 1

        if m == 3000:
            images_aug = seq.augment_images(imageList)
            for n in range(len(images_aug)):
                cv2.imwrite(TRAIN_AUG_DIR + "aug_" + str(k+n) + ".jpg",images_aug[n])
            imageList.clear()
        #写入到文件中
        if m == 5000:
            images_aug = seq.augment_images(imageList)
            for n in range(len(images_aug)):
                cv2.imwrite(TRAIN_AUG_DIR + "aug_" + str(3000+n) + ".jpg",images_aug[n])
            imageList.clear()

    #增强到5000
    TRAIN_DIR = "../data/normal_68/"
    TRAIN_AUG_DIR = "../data/train/normal/"
    import cv2
    import os
    from random import choice
    from imgaug import augmenters as iaa
    import imgaug as ia

    seq = iaa.Sequential([
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                iaa.Multiply((0.9, 1.1)),
            ])

    normal_list = os.listdir(TRAIN_AUG_DIR)

    k = len(normal_list)
    imageList = []
    m = k
    while m < 5000:
        image = cv2.imread(TRAIN_AUG_DIR + choice(normal_list))
        imageList.append(image)
        m += 1

        if m == 3000:
            images_aug = seq.augment_images(imageList)
            for n in range(len(images_aug)):
                cv2.imwrite(TRAIN_AUG_DIR + "aug_" + str(k+n) + ".jpg",images_aug[n])
            imageList.clear()
        #写入到文件中
        if m == 5000:
            images_aug = seq.augment_images(imageList)
            for n in range(len(images_aug)):
                cv2.imwrite(TRAIN_AUG_DIR + "aug_" + str(3000+n) + ".jpg",images_aug[n])
            imageList.clear()
            
    #建立模型
    model = inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False,input_shape=(600,800,3))
    x = model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(2,activation='softmax',name='predictions')(x)
    densenet_model = Model(inputs = model.input, outputs = predictions)
    #优化器为adam
    adam = Adam(lr=0.001)

    print("正常",len(os.listdir("../data/train/normal")))
    print("异常",len(os.listdir("../data/train/bad")))

    image_generator = ImageDataGenerator(
            horizontal_flip=True,
            vertical_flip=True,
            rescale=1./255)
    train_generator = image_generator.flow_from_directory(directory='../data/train/',
                                                         target_size=(600,800),
                                                         batch_size=8,
                                                         class_mode='categorical')


    class CustomModelCheckpoint(keras.callbacks.Callback): 
        def __init__(self, model, path):
            self.model = model
            self.path = path
            self.best_loss = np.inf
            self.best_auc = 0

        def on_epoch_end(self, epoch, logs=None): 
            start = time.time()
            y_val_predict_all = []
            y_val = []
            #读取验证集normal的一张图像
            for i in os.listdir("../data/val/normal"):
                img = cv2.imread("../data/val/normal/" + i)
                #转换为rgb
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)
                batch_x = np.zeros((1,600,800,3),backend.floatx())
                batch_x[0] = img
                y_val_predict = self.model.predict(batch_x/255.,verbose=0,batch_size=8)
                #取其中可能性最大的为该图片的有问题与否可能性
                y_val_predict = np.max(y_val_predict,axis=0)[0]
                y_val_predict_all.append(y_val_predict)
                #normal 可能性应该为0
                y_val.append(0)
            print("正常的预测完成")

            #读取验证集bad的一张图像
            for i in os.listdir("../data/val/bad"):
                img = cv2.imread("../data/val/bad/" + i)
                #转换为rgb
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)
                batch_x = np.zeros((1,600,800,3),backend.floatx())
                batch_x[0] = img

                y_val_predict = self.model.predict(batch_x/255.,verbose=0,batch_size=8)

                #取其中可能性最大的为该图片的有问题与否可能性
                y_val_predict = np.max(y_val_predict,axis=0)[0]
                y_val_predict_all.append(y_val_predict)
                #bad 可能性应该为1
                y_val.append(1)
            print("瑕疵的预测完成")

            print("验证集验证完成")

            end = time.time()

            print("用时:",end - start)


            from sklearn import metrics
            val_auc = metrics.roc_auc_score(y_val,y_val_predict_all)
            print('val_auc:',val_auc)

            abc = np.array(y_val_predict_all)
            abc[abc>=0.5] = 1
            abc[abc<0.5] = 0
            acc = np.sum(abc == np.array(y_val))/len(y_val)
            print("accuracy:",acc)

            if val_auc > 0.99 and acc > 0.99: 
                print("\n saving model")
                self.model.save(self.path)
                self.best_auc = val_auc
                #进行test

                y_val_predict_all = []
                csv_list=[]
                y_val_predict = []

                #读取验证集normal的一张图像
                for name in os.listdir("../data/xuelang_round1_test_a_20180709"):
                    img = cv2.imread("../data/xuelang_round1_test_a_20180709/" + name)
                    #转换为rgb
                    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)
                    batch_x = np.zeros((1,600,800,3),backend.floatx())
                    batch_x[0] = img

                    y_val_predict = self.model.predict(batch_x/255.,verbose=1,batch_size=8)
                    y_val_predict = np.max(y_val_predict,axis=0)[0]
                    csv_list.append((name,y_val_predict))
                print("预测完成")

                import datetime
                csv_name = "../submit/submit_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S')+".csv"
                with open(csv_name,"a+",newline='') as csv_file:
                    fieldnames = ['filename', 'probability']
                    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                    writer.writeheader()
                    for img_name, prob in csv_list:
                        #print(np.round(np.clip(prob,0.000001,0.999999),6))
                        writer.writerow({'filename': img_name, 'probability': np.round(np.clip(prob,0.000001,0.999999),6)})

    from keras.utils import multi_gpu_model
    from keras.optimizers import SGD
    # Replicates `model` on 8 GPUs.
    # This assumes that your machine has 8 available GPUs.
    parallel_model = multi_gpu_model(densenet_model, gpus=2)
    parallel_model.compile(loss='categorical_crossentropy',optimizer=adam,metrics = ['accuracy'])
    history = parallel_model.fit_generator(train_generator,verbose=1,epochs=100,callbacks=[CustomModelCheckpoint(densenet_model,'checkpoint.h5')])

In [ ]:
main()